In [1]:
import re
import csv
import sys, random
import pandas as pd
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW

###   Random Sampling을 통한 new train dataset 구성

In [2]:
def random_positive(path,wr,duplicate_check):
    numbers = list(range(1,301))
    random_num = random.choice(numbers)
    random_dir = "problem"+str(random_num).zfill(3)
    code = []
    tuple_set = []
    for _ in range(2):
        random_file = random.randint(1,150)
        num = 0
        while (random_num,random_file) in duplicate_check and len(numbers)>0:
            random_file = random.randint(1,150)
            num += 1
            if num > 100:
                numbers.remove(random_num)
                random_num = random.choice(numbers)
                num=0
        tuple_set.append((random_num,random_file))
        file_name = path+"/"+random_dir+"/"+random_dir+"_"+str(random_file)+".py"

        with open(file_name,'r') as file:
            code.append("".join(file.readlines()))
    wr.writerow([code[0],code[1],1])
    return tuple_set


def random_negative(path,wr,duplicate_check):
    code = []
    tuple_set = []
    random_num1 = random.randint(1,300)
    random_file1 = random.randint(1,150)
    num1 = 0
    while (random_num1,random_file1) in duplicate_check:
        random_file1 = random.randint(1,150)
        num1 += 1
        if num1>100:
            random_num1 = random.randint(1,300)
            num1=0
    tuple_set.append((random_num1,random_file1))
    
    while(True):
        random_num2 = random.randint(1,300)
        if random_num2 != random_num1:
            break
    random_file2 = random.randint(1,150)
    num2=0
    while (random_num2,random_file2) in duplicate_check:
            random_file2 = random.randint(1,150)
            num2 +=1
            if num2>100:
                while(True):
                    random_num2 = random.randint(1,300)
                    if random_num2 != random_num1:
                        break
    tuple_set.append((random_num2,random_file2))
    
    random_dir1 = "problem"+str(random_num1).zfill(3)
    random_dir2 = "problem"+str(random_num2).zfill(3)
    
    file_name1 = path+"/"+random_dir1+"/"+random_dir1+"_"+str(random_file1)+".py"
    file_name2 = path+"/"+random_dir2+"/"+random_dir2+"_"+str(random_file2)+".py"
    with open(file_name1,'r') as file:
        code.append("".join(file.readlines()))
    with open(file_name2,'r') as file:
        code.append("".join(file.readlines()))
    
    wr.writerow([code[0],code[1],0])
    return tuple_set


if __name__ == "__main__":
    path = str(sys.path[0]) + "/code/"
    f = open('new_train_sample_5.csv','w',newline='\n') # 생성된 파일을 아래 코드에서 train dataset으로 불러옵니다.
    wr = csv.writer(f)
    wr.writerow(['code1','code2','similar'])
    row = 0
    random_list = [random.randint(0,1) for r in range(45088)]
    duplicate_check_positive = []
    duplicate_check_negative = []

    while(row<45088): #pair 개수
        if row%100==0: print(row)
        if random_list[row] == 0:
            duplicate_check_positive += random_positive(path,wr,duplicate_check_positive)
        elif random_list[row] == 1:
            duplicate_check_negative += random_negative(path,wr,duplicate_check_negative)
        row +=1

    f.close()

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18


KeyboardInterrupt



In [3]:
import csv

# 두 파이썬 코드 파일 경로
file_path1 = "baekjoon68367943(1).py"
file_path2 = "baekjoon68367943(2).py"
is_similar = 1  # 유사한 코드인 경우 1, 유사하지 않은 경우 0

# 첫 번째 파일 읽기
with open(file_path1, 'r') as file1:
    code1 = file1.read()

# 두 번째 파일 읽기
with open(file_path2, 'r') as file2:
    code2 = file2.read()

# CSV 파일 열기
with open('new_train_sample_5.csv', 'a', newline='') as file:
    writer = csv.writer(file)

    # 샘플 추가
    writer.writerow([code1, code2, is_similar])

print("샘플이 CSV 파일에 추가되었습니다.")

샘플이 CSV 파일에 추가되었습니다.


### 모델 구현 및 학습, 추론


*   Load Train dataset

In [12]:
train = pd.read_csv('new_train_sample_5.csv', encoding = 'utf-8')
df = pd.DataFrame(train)
df.tail(5)
# Dataframe의 맨 마지막 row(44949)를 보면 Own Data가 추가된 것을 볼 수 있음.

,code1,code2,similar
44945,"n, m = map(int, input().split())\nc = list(map...",a = input()\nb = a.split(' ')\nc = int(b[0])\n...,0
44946,print('aA'[input()<'a'])\n,alpha = str(input())\nif alpha.isupper():\n a...,1
44947,"N=int(input())\nA=map(int, input().split())\nP...",import sys\nimport heapq\n\nsys.setrecursionli...,0
44948,"a, b, c = map(int, raw_input().split())\n\nn =...",#!/usr/bin/env python3\nimport sys\nfrom decim...,0
44949,# https://www.acmicpc.net/source/68367943\nstr...,# https://www.acmicpc.net/source/68367943\nstr...,1



*   Define Model, tokenizer


In [37]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

batch_size = 16
epoch_num = 16
MAX_LEN = 512
learning_rate = 2e-4

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

* Preprocessing



In [38]:
# 전처리는 사실상 토큰화하는 것까지 포함하고 있으나 여기서는 편의상 따로 함수를 만들어 진행했습니다.

def preprocess(df,file_name):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv(file_name)
  
def tokenized(examples):
  return tokenizer(examples['code1'],examples['code2'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [39]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['Unnamed: 0','code1','code2'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='similar',new_column_name='labels')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/44950 [00:00<?, ? examples/s]

* 데이터 나누기

In [40]:
encoded_dataset = encoded_dataset.train_test_split(0.1,seed=100)

* optimizers 설정

In [41]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8, 
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [42]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [43]:
#metric = load_metric("accuracy")
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [44]:
args = TrainingArguments("test", save_strategy="epoch",evaluation_strategy="epoch",logging_strategy="epoch", 
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model="accuracy",load_best_model_at_end=True)

trainer = Trainer(model,args,train_dataset=encoded_dataset['train'],eval_dataset=encoded_dataset['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [45]:
gc.collect()
torch.cuda.empty_cache()

* 학습

In [46]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.704100,0.705160,0.502336
2,0.701300,0.699441,0.497664
3,0.699400,0.694478,0.502336
4,0.696000,0.693507,0.502336
5,0.695400,0.693135,0.502336
6,0.695200,0.696580,0.497664
7,0.694700,0.693481,0.502336
8,0.694500,0.693274,0.502336
9,0.694400,0.693892,0.497664
10,0.694300,0.693236,0.502336


/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqu

TrainOutput(global_step=5072, training_loss=0.6958366117266827, metrics={'train_runtime': 2993.4713, 'train_samples_per_second': 216.231, 'train_steps_per_second': 1.694, 'total_flos': 1.703065239134208e+17, 'train_loss': 0.6958366117266827, 'epoch': 16.0})

In [47]:
trainer.evaluate()

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.7051596641540527,
 'eval_accuracy': 0.5023359288097886,
 'eval_runtime': 9.4347,
 'eval_samples_per_second': 476.431,
 'eval_steps_per_second': 3.816,
 'epoch': 16.0}

# Model save

In [89]:
model.save_pretrained("code_similarity_model")
tokenizer.save_pretrained("code_similarity_model")

('code_similarity_model/tokenizer_config.json',
 'code_similarity_model/special_tokens_map.json',
 'code_similarity_model/vocab.json',
 'code_similarity_model/merges.txt',
 'code_similarity_model/added_tokens.json',
 'code_similarity_model/tokenizer.json')

# Model 불러온 후 적용

In [90]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# 불러온 모델과 토크나이저의 경로
model_path = "code_similarity_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Python 코드가 들어있는 두 개의 py 파일 경로
# 아래 백준사이트에서 코드를 가져옴.
# https://www.acmicpc.net/status?from_mine=1&problem_id=17609&user_id=chan4im
# GitHub의 경우, PostProcessing_Test_Python_files 디렉토리내부에 존재
file_path1 = "code1.py"
file_path2 = "code2.py"

# 두 개의 py 파일을 읽어와서 코드를 가져옴
with open(file_path1, 'r') as file1:
    code1 = file1.read()

with open(file_path2, 'r') as file2:
    code2 = file2.read()

# 코드 유사성 평가를 위한 입력 생성
inputs = tokenizer(code1, code2, padding=True, max_length=MAX_LEN, truncation=True, return_tensors="pt")
outputs = model(**inputs)
predictions = outputs.logits

In [91]:
print(predictions)

tensor([[-2.9283,  2.6643]], grad_fn=<AddmmBackward0>)


In [92]:
import torch
import torch.nn.functional as F

# 로짓 값
logits = predictions[0]

# Softmax함수를 사용하여 확률로 변환
probabilities = F.softmax(logits, dim=0)

# 유사성(%)얻기
similarity_percentage = probabilities[1].item() * 100

print(f"유사성: {similarity_percentage:.2f}%")


유사성: 99.63%
